# To install PySpark
https://gist.github.com/tommycarpi/f5a67c66a8f2170e263c

Put export PYSPARK_DRIVER_PYTHON=ipython3

Possible problems in take(3):
http://stackoverflow.com/questions/30518362/how-do-i-set-the-drivers-python-version-in-spark

In [1]:
sc

# Import

In [2]:
# Import the fucking useful libraries =)
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import itertools
from pyspark.mllib.recommendation import ALS
import math

%matplotlib inline
%load_ext autoreload
%autoreload 2

# There's a lot of columns in the DF. 
# Therefore, we add this option so that we can see more columns
pd.options.display.max_columns = 100

# Load and prepare training data

In [3]:
train = pd.read_csv('../data/data_train.csv')

In [4]:
train['UserID'] = train['Id'].apply(lambda x: int(x.split('_')[0][1:]))
train['MovieID'] = train['Id'].apply(lambda x: int(x.split('_')[1][1:]))
train['Rating'] = train['Prediction']
train = train.drop(['Id', 'Prediction'], axis=1)

In [5]:
train.head()

,UserID,MovieID,Rating
0,44,1,4
1,61,1,3
2,67,1,4
3,72,1,3
4,86,1,5


In [6]:
train.shape

(1176952, 3)

In [8]:
# First, we transform it using sqlContect
train_sql = sqlContext.createDataFrame(train)
train_rdd = train_sql.rdd
train_rdd.take(3)

[Row(UserID=44, MovieID=1, Rating=4),
 Row(UserID=61, MovieID=1, Rating=3),
 Row(UserID=67, MovieID=1, Rating=4)]

In [10]:
train_rdd

MapPartitionsRDD[11] at javaToPython at NativeMethodAccessorImpl.java:-2

# Training phase

In [ ]:
def computeRMSE(model, data, prediction):
    predictions = model.predictAll(data).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = prediction.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    return error